# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Webservice, Model
from azureml.train.automl import AutoMLConfig
import pandas as pd


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.subscription_id, sep = '\n')

quick-starts-ws-131770
aml-quickstarts-131770
6b4af8be-9931-443e-90f6-c4c34a1f9737


In [10]:
# choose a name for experiment
experiment_name = 'capstone-twitter-exp'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-twitter-exp,quick-starts-ws-131770,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu-cluster"

try:
    new_cluster=ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Existing cluster detected, make use of it!")
    
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='dedicated',min_nodes=None,max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True)         
    print("Cluster is ready")

Existing cluster detected, make use of it!


In [13]:
datastore = ws.get_default_datastore()

datastore.upload(src_dir='./', target_path='dataset')

Uploading an estimated of 3 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 3
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-25-19-49-46.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl-checkpoint2020-11-25-19-49-46.ipynb, 2 files out of an estimated total of 3
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_e427b645b94849779e1f29316c2e074c

In [14]:
dataset = Dataset.get_by_name(ws, name="twitter-user")
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/12-25-2020_074602_UTC/gender-classifier.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ac8ff0f1-1cd2-4330-88ea-a4bd244c2529",
    "name": "twitter-user",
    "version": 1,
    "description": "External dataset from kaggle",
    "workspace": "Workspace.create(name='quick-starts-ws-131770', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-131770')"
  }
}

In [15]:
dataset.take(5).to_pandas_dataframe() 

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Perkins, Yahoo!, Sprint PCS, AirTouch, Air Force. Stanford GSB, UVa. Dad, Husband, Brother. Golfer.",202,Column113,0084B4_30,douggarland,Column116,https://pbs.twimg.com/profile_images/259703936/IMG_8444_normal.JPG,0_31,C0DEED_32
0,"#Islam #Muslim #muslims""",None,52230,10/26/15 12:40,6.5873E+17,ksa,Pacific Time (US & Canada)\r815719253,FALSE,finalized,3,...,3,10/26/15 23:12,male,1,yes,1,12/5/12 6:18,"""Award Winning Author of the paranormal Romanc...",Hath No Fury technology writer,None
1,"http://t.co/LmMiOwenQQ #ASMSG""",103,None,DD2E44,HalRappaport,None,https://pbs.twimg.com/profile_images/363652832...,0,FFFFFF,All the #magic in Hath No FURY is based on REA...,...,None,None,None,None,None,None,None,None,None,None
2,Me November 1st: it's my birthday month,"bow down to me""",None,4965,10/26/15 12:40,6.5873E+17,"Algonac & Mt. Pleasant, MI",Eastern Time (US & Canada)\r815719259,FALSE,finalized,...,None,None,None,None,None,None,None,None,None,None
3,"All I Kno Is Grind Wea I Come From ‰Ïö""",49,None,0084B4,300Stain,None,https://pbs.twimg.com/profile_images/656370495...,0,FFFFFF,Did Alot Up In The Past Ion Wont Back,...,None,None,None,None,None,None,None,None,None,None
4,capri eternal...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [19]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             experiment_timeout_minutes=30,
                             primary_metric='AUC_weighted',
                             n_cross_validations=5,
                             training_data=dataset,
                             max_concurrent_iterations=4,
                             max_cores_per_iteration=4,
                             compute_target=new_cluster,
                             label_column_name='gender',
                             featurization='auto',
                             enable_voting_ensemble=True,
                             enable_early_stopping=True
)

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_41647fcc-8859-4042-82a7-246ba8ebfa70

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a fal


****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=================================+=================================+
|_unit_id                         |text                             |
|_golden                          |text                             |
|_trusted_judgments               |categorical_hash                 |
|_last_judgment_at                |categorical_hash                 |
|gender:confidence                |text     


****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         3   MaxAbsScaler RandomForest                      0:02:00       0.5327    0.5327
         4   MaxAbsScaler RandomForest                      0:04:28       0.6035    0.6035
         2   MaxAbsScaler RandomForest                      0:04:33       0.6219    0.6219
         6   MaxAbsScaler ExtremeRandomTrees                0:03:22      

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name)
    print(metric)

In [ ]:
# Wait for the best model explanation run to complete

from azureml.core.run import Run
model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

In [ ]:
#TODO: Save the best model

best_run.register_model(model_name='automl_model',model_path='/outputs',properties={'AUC_weighted':best_run_metrics['AUC_weighted']},tags={'Training context':'Auto ML'})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "bmData", 'type': "automl_classification"}, 
                                               description = 'sample service for Automl Classification')

aci_service_name = 'automl-sample-bankmarketing-all'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service